## Library Import

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Load

In [ ]:
# 데이터 경로 설정
data_path = "~/movie/data/train"

# 전체 학습 데이터 불러오기
train_ratings = pd.read_csv(os.path.join(data_path, "train_ratings.csv"))

# 아이템 side information 불러오기
years = pd.read_csv(os.path.join(data_path, "years.tsv"), sep="\t")
writers = pd.read_csv(os.path.join(data_path, "writers.tsv"), sep="\t")
titles = pd.read_csv(os.path.join(data_path, "titles.tsv"), sep="\t")
genres = pd.read_csv(os.path.join(data_path, "genres.tsv"), sep="\t")
directors = pd.read_csv(os.path.join(data_path, "directors.tsv"), sep="\t")

In [ ]:
# side information을 하나의 아이템 데이터프레임으로 병합
item_df = pd.merge(titles, years, on="item", how="left")
item_df = pd.merge(item_df, directors, on="item", how="left")
item_df = pd.merge(item_df, writers, on="item", how="left")
item_df = pd.merge(item_df, genres, on="item", how="left")

In [ ]:
item_df

## Data Preprocessing

### 1. `year`

결측치를 `title`에 포함된 연도 정보를 활용해 대체한다.

In [ ]:
# year 결측치 확인
item_df[item_df["year"].isna()]

In [ ]:
# title에서 괄호 안 연도 추출해 year 결측치 대체
item_df["year"] = item_df["year"].fillna(
    item_df["title"].str.extract(r"\((\d{4})\)", expand=False)  # 괄호 안 네 자리 숫자를 추출하는 정규표현식
).astype("int64")

# 결과 확인
print(item_df[item_df["year"].isna()])  # 여전히 NaN인 경우 확인

### 2. `title`

#### 2.1 같은 `title`, 다른 `item` 처리
같은 영화 중 다른 item 값을 갖는 데이터 중 결측치가 없는 item을 기준으로 통일시킨다.

In [ ]:
delete_title = (titles["title"].value_counts() > 1).index[0]
print("Before droping the indices")
display(item_df[item_df["title"] == delete_title])

# 중복된 title을 갖는 item 제거
item_df = item_df.drop(index=[13507, 13508])
print("\nAfter drop the indices")
display(item_df[item_df["title"] == delete_title])

In [ ]:
# train_ratings에서 item 값을 변경하고자 하는 인덱스 목록 추출
idx = train_ratings[(train_ratings["item"] == 64997)].index

# 원하는 item 값으로 변경
train_ratings.loc[idx, "item"] = 34048

In [ ]:
# 결측치 대체가 잘 됐는지 확인
train_ratings[(train_ratings["item"] == 64997)]

#### 2.2 `title` 재구성

현재 `title`은 _**"영문 제목 (a.k.a. 별칭) (원어 제목) (연도)"**_ 순으로 구성되어 있다. 정규표현식을 활용하여 다음 순서대로 `title`을 재구성한다.

1. 따옴표(”, ‘) 제거
2. 영문 제목만 선택
3. 관사 위치 재조정: "~, The"를 "The ~"로 변경
4. 특수문자 삭제
5. 소문자로 통일

In [ ]:
def preprocess_title(title):
    # 1. 따옴표(”, ‘) 제거
    title = re.sub(r'^[\'"](.*)[\'"]$', r'\1', title)
    
    # 2. 영문 제목만 추출
    title = re.match(r'^[^(]+', title).group().strip() if re.match(r'^[^(]+', title) else title
    
    # 3. "~, The", "~, A", "~, An" 형태를 "The ~", "A ~", "An ~"으로 변경
    title = re.sub(r'^(.*),\s(The|A|An)$', r'\2 \1', title)
    
    # 4. 특수문자 제거
    title = re.sub(r'[^a-zA-Z0-9\s]', '', title)
    
    # 5. 소문자로 변환
    title = title.lower()
    
    return title

In [ ]:
# 현재 error 발생
item_df["title"] = preprocess_title(item_df["title"])

### 3. `director`

In [ ]:
item_director_counts = directors.groupby('item')['director'].nunique().reset_index()
item_director_counts.columns = ['item', 'director_count']

director_counts = item_director_counts.groupby('director_count').count().reset_index()
director_counts.columns = ['director_count', 'count']
director_counts["count"].sum()

### 4. `writer`

### 5. `genre`

## Feature Engineering

In [ ]:
# train_ratings와 item_df를 최종 병합
train_df = pd.merge(train_ratings, item_df, on="item", how="left")

### 파생변수 생성